In [1]:
rm(list = ls())
setwd("/mnt/c/git_repos/iORD_hepatitis/")
require(tidyverse)
require(data.table)
require(lubridate)
require(foreach)
require(doParallel)
require(funtimes)
require(tidyquant)

Loading required package: tidyverse

Warning message:
“package ‘tidyverse’ was built under R version 4.1.3”
Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0     ✔ purrr   1.0.1
✔ tibble  3.1.8     ✔ dplyr   1.1.0
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 1.0.0
Warning message:
“package ‘ggplot2’ was built under R version 4.1.3”
Warning message:
“package ‘tibble’ was built under R version 4.1.3”
Warning message:
“package ‘tidyr’ was built under R version 4.1.3”
Warning message:
“package ‘readr’ was built under R version 4.1.3”
Warning message:
“package ‘purrr’ was built under R version 4.1.3”
Warning message:
“package ‘dplyr’ was built under R version 4.1.3”
Warning message:
“package ‘stringr’ was built under R version 4.1.3”
Warning message:
“package ‘forcats’ was built under

### Load data

In [2]:
df <- fread("data/IORD_ASHep-UE_34_20230120_LIMS.csv") %>%
    mutate(AttendanceStartDate = as.Date(AttendanceStartDate),
           CollectionDateTime = as.Date(CollectionDateTime)) %>%
    filter(AttendanceStartDate < as.Date("2023-01-01"))

age_df <- fread("data/IORD_ASHep-UE_34_20230120_Attendances.csv") %>%
    filter(AttendanceStartDate < as.Date("2023-01-01")) %>%
    mutate(AttendanceStartDate = as.Date(AttendanceStartDate),
           AttendanceEndDate = as.Date(AttendanceEndDate),
           LinkedBirthmonth = as.Date(LinkedBirthmonth)) %>%
    mutate(AttendanceMonth = cut(AttendanceStartDate, breaks = "month"),
           AttendanceTwoMonth = cut(AttendanceStartDate, breaks = "2 months")) %>%
    mutate(AttendanceMonth = as.Date(AttendanceMonth),
           AttendanceTwoMonth = as.Date(AttendanceTwoMonth)) %>%
    mutate(age_upon_presentation = time_length(AttendanceStartDate -
                                               LinkedBirthmonth, "year")) %>%
    mutate(age_upon_presentation = floor(age_upon_presentation)) %>% 
    mutate(age_group = case_when(age_upon_presentation < 7 ~ "<7",
                                 age_upon_presentation >= 6 & age_upon_presentation <= 15 ~ "7-15",
                                 age_upon_presentation >= 16 ~ ">15")) %>%
    arrange(AttendanceStartDate)

month_df <- df %>%
    mutate(CollectionMonth = cut(CollectionDateTime, breaks = "month")) %>%
    mutate(CollectionTwoMonths = cut(CollectionDateTime, breaks = "2 months")) %>%
    mutate(CollectionMonth = as.Date(CollectionMonth),
           CollectionTwoMonths = as.Date(CollectionTwoMonths)) %>%
    left_join(age_df) %>%
    arrange(AttendanceStartDate)

admission_df <- fread("data/IORD_ASHep-UE_34_20230120_InpatientSpells.csv") %>%
    filter(AttendanceStartDate < as.Date("2023-01-01")) %>%
    mutate(AttendanceStartDate = as.Date(AttendanceStartDate))

test_df <- fread("data/IORD_ASHep-UE_34_20230120_LIMS.csv") %>%
    mutate(CollectionDateTime = as.Date(CollectionDateTime),
           AttendanceStartDate = as.Date(AttendanceStartDate)) %>%
    filter(AttendanceStartDate < as.Date("2023-01-01"))

test_parsed <- test_df %>%
    distinct(ClusterID, AttendanceStartDate, TestName, .keep_all = T) %>%
    separate(RefRange, into = c("LLN", "ULN"), sep = "-") %>%
    mutate(ULN = as.numeric(ULN)) %>%
    select(-CollectionDateTime, -ReceiveDateTime) %>%
    mutate(Value = as.numeric(Value)) %>%
    mutate(Value = ifelse(is.na(Value), 0, Value)) %>%
    mutate(value_class = case_when(Value <= ULN ~ "Normal <= ULN",
                                 Value > ULN & Value <= 2 * ULN ~ "Mild (1-2x ULN)",
                                 Value > 2 * ULN & Value <= 5 * ULN  ~ "Moderate (2-5x ULN)",
                                 Value > 5 * ULN ~ "Severe (>5x ULN)")) %>%
    mutate(value_class = factor(value_class, c("Normal <= ULN", 
                                               "Mild (1-2x ULN)", 
                                               "Moderate (2-5x ULN)",
                                               "Severe (>5x ULN)"))) 

diag_df <- fread("data/IORD_ASHep-UE_34_20230120_DiagnosisCodes.csv") %>%
    left_join(fread("data/IORD_ASHep-UE_34_20230120_InpatientSpells.csv")) %>%
    mutate(AttendanceStartDate = as.Date(AttendanceStartDate)) %>%
    left_join(age_df) %>%
    mutate(age_upon_presentation = floor(age_upon_presentation)) %>% 
    mutate(age_group = case_when(age_upon_presentation < 7 ~ "<7",
                                 age_upon_presentation >= 6 & age_upon_presentation <= 15 ~ "7-15",
                                 age_upon_presentation >= 16 ~ ">15")) %>%
    mutate(AttendanceMonth = cut(AttendanceStartDate, breaks = "month"),
           AttendanceTwoMonth = cut(AttendanceStartDate, breaks = "2 months")) %>%
    mutate(AttendanceMonth = as.Date(AttendanceMonth),
           AttendanceTwoMonth = as.Date(AttendanceTwoMonth)) %>%
    arrange(AttendanceMonth) %>%
    distinct(ClusterID, AttendanceStartDate, DiagCode)

Joining with `by = join_by(ClusterID, AttendanceStartDate)`
Warning message in left_join(., age_df):
“Each row in `x` is expected to match at most 1 row in `y`.
ℹ Row 42 of `x` matches multiple rows.
ℹ If multiple matches are expected, set `multiple = "all"` to silence this warning.”
Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `ULN = as.numeric(ULN)`.
Caused by warning:
! NAs introduced by coercion”
Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Value = as.numeric(Value)`.
Caused by warning:
! NAs introduced by coercion”
Joining with `by = join_by(ClusterID, SpellID)`
Warning message in left_join(., fread("data/IORD_ASHep-UE_34_20230120_InpatientSpells.csv")):
“Each row in `x` is expected to match at most 1 row in `y`.
ℹ Row 148 of `x` matches multiple rows.
ℹ If multiple matches are expected, set `multiple = "all"` to silence this warning.”
Joining with `by = join_by(ClusterID, AttendanceStartDate)`
Warning message in left_join(., age_df):

### Dataset range

In [3]:
total_number <- age_df %>%
    distinct(ClusterID, AttendanceStartDate) %>%
    nrow()

total_number

# Sex breakdown
count_df <- age_df %>%
    distinct(ClusterID, AttendanceStartDate, .keep_all = T) %>%
    group_by(age_group) %>%
    summarise(n_total = n())

age_df %>%
    distinct(ClusterID, AttendanceStartDate, LinkedSex) %>%
    group_by(LinkedSex) %>%
    summarise(n = n())

age_df %>%
    distinct(ClusterID, AttendanceStartDate, LinkedSex, .keep_all = T) %>%
    group_by(LinkedSex, age_group) %>%
    summarise(n = n()) %>%
    left_join(count_df) %>%
    mutate(prop = round(n / n_total * 100, 2))

# Age summary stats
age_df %>%
    distinct(ClusterID, AttendanceStartDate, age_upon_presentation) %>%
    summarise(median = median(age_upon_presentation),
              low_IQR = quantile(age_upon_presentation, prob = c(0.25)),
              high_IQR = quantile(age_upon_presentation, prob = c(0.75)))

age_df %>%
    distinct(ClusterID, AttendanceStartDate, .keep_all = T) %>%
    group_by(age_group) %>%
    summarise(median = median(age_upon_presentation),
              low_IQR = quantile(age_upon_presentation, prob = c(0.25)),
              high_IQR = quantile(age_upon_presentation, prob = c(0.75)))

# Ethnicity 
# age_df %>%
#     distinct(ClusterID, AttendanceStartDate, .keep_all = T) %>%
#     group_by(EthnicGroupCode) %>%
#     summarise(n = n())

# IMD score
age_df %>%
    distinct(ClusterID, AttendanceStartDate, .keep_all = T) %>%
    mutate(IMDScore = as.numeric(IMDScore)) %>%
    filter(!is.na(IMDScore)) %>%
    group_by(age_group) %>%
    summarise(median_IMD2 = quantile(IMDScore, prob = c(0.2)),
             median_IMD4 = quantile(IMDScore, prob = c(0.4)),
             median_IMD6 = quantile(IMDScore, prob = c(0.6)),
             median_IMD8 = quantile(IMDScore, prob = c(0.8)))

# Attendance per month
age_df %>%
    distinct(ClusterID, AttendanceStartDate, .keep_all = T) %>%
    group_by(AttendanceMonth) %>%
    summarise(n_admit = n()) %>%
    summarise(median_admit = median(n_admit))

[1] 903433

LinkedSex,n
<chr>,<int>
F,461632
I,16
M,441780
U,5


`summarise()` has grouped output by 'LinkedSex'. You can override using the `.groups` argument.
Joining with `by = join_by(age_group)`


LinkedSex,age_group,n,n_total,prop
<chr>,<chr>,<int>,<int>,<dbl>
F,7-15,39498,86217,45.81
F,<7,30446,70962,42.90
F,>15,391688,746254,52.49
I,7-15,4,86217,0.00
I,>15,12,746254,0.00
M,7-15,46715,86217,54.18
M,<7,40516,70962,57.10
M,>15,354549,746254,47.51
U,>15,5,746254,0.00


median,low_IQR,high_IQR
<dbl>,<dbl>,<dbl>
44,22,69


age_group,median,low_IQR,high_IQR
<chr>,<dbl>,<dbl>,<dbl>
7-15,11,9,13
<7,3,2,5
>15,53,32,74


Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `IMDScore = as.numeric(IMDScore)`.
Caused by warning:
! NAs introduced by coercion”


age_group,median_IMD2,median_IMD4,median_IMD6,median_IMD8
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
7-15,5.792,8.982,13.215,20.4810
<7,5.904,9.156,13.619,21.4902
>15,6.061,9.193,13.444,20.2410


median_admit
<dbl>
11023


In [8]:
age_df %>%
    distinct(ClusterID, AttendanceStartDate, .keep_all = T) %>%
    group_by(age_group) %>%
    summarise(n = n()) %>% 
    mutate(n / sum(n))

age_group,n,n/sum(n)
<chr>,<int>,<dbl>
7-15,86217,0.09543264
<7,70962,0.07854705
>15,746254,0.82602030


### Admissions

In [21]:
admit_attendance_df

ClusterID,AttendanceStartDate,AttendanceEndDate,LocationAttended,IMDScore,EthnicGroupCode,LinkedSex,LinkedBirthmonth,LinkedDeathdate,SpineCheckDate,⋯,age_group,SpellID,AdmissionDate,DischargeDate,DischargeDestinationCode,SpellIncludesCriticalCareAdmission,duration,is_admitted,is_icu,is_death
<int>,<date>,<dbl>,<chr>,<chr>,<chr>,<chr>,<date>,<date>,<chr>,⋯,<chr>,<chr>,<dttm>,<chr>,<int>,<int>,<drtn>,<lgl>,<lgl>,<lgl>
1716156,2016-03-01,16861,J-WD JR_Chi CDU,27.969,A,F,2007-10-01,2030-01-14,2023-01-14 14:00:08.433,⋯,7-15,NA,NA,NA,NA,NA,NA days,FALSE,FALSE,FALSE
2077289,2016-03-01,16861,J-ED,5.173,A,M,1946-03-01,2030-01-14,2023-01-11 14:00:09.793,⋯,>15,3734500,2016-03-01 13:54:00,2016-03-07 15:10:00.000,19,0,6 days,TRUE,TRUE,FALSE
1458610,2016-03-01,16861,J-ED,25.391,Z,M,1995-12-01,2030-01-14,2023-01-16 14:00:15.287,⋯,>15,NA,NA,NA,NA,NA,NA days,FALSE,FALSE,FALSE
1981154,2016-03-01,16861,J-ED,20.047,L,M,2002-12-01,2030-01-14,2023-01-13 11:00:10.767,⋯,7-15,NA,NA,NA,NA,NA,NA days,FALSE,FALSE,FALSE
3950138,2016-03-01,16862,J-ED,7.879,A,M,1984-12-01,2030-01-14,2023-01-11 17:00:08.747,⋯,>15,NA,NA,NA,NA,NA,NA days,FALSE,FALSE,FALSE
6834862,2016-03-01,16862,J-WD EAU,9.802,A,F,1924-02-01,2019-09-19,2021-12-11 11:00:06.790,⋯,>15,3735180,2016-03-01 14:24:00,2016-03-02 16:10:00.000,19,0,1 days,TRUE,TRUE,FALSE
5079218,2016-03-01,16862,J-ED,5.41,A,M,1932-10-01,2019-04-08,2021-12-11 11:00:06.790,⋯,>15,3735888,2016-03-01 19:50:00,2016-03-04 15:00:00.000,19,0,3 days,TRUE,TRUE,FALSE
8098274,2016-03-01,16861,J-ED,11.728,A,F,2004-04-01,2030-01-14,2023-01-10 17:00:10.237,⋯,7-15,NA,NA,NA,NA,NA,NA days,FALSE,FALSE,FALSE
5302,2016-03-01,16861,J-ED,8.88,C,M,2004-11-01,2030-01-14,2023-01-19 11:00:09.643,⋯,7-15,NA,NA,NA,NA,NA,NA days,FALSE,FALSE,FALSE


In [22]:
admit_attendance_df <- age_df %>%
    left_join(admission_df) %>%
    distinct(ClusterID, AttendanceStartDate, .keep_all = T) %>%
    mutate(LinkedDeathdate = ifelse(LinkedDeathdate == "NULL", "2030-01-14 14:00:08.433", LinkedDeathdate),
           duration = as.Date(DischargeDate) - as.Date(AdmissionDate)) %>%
    mutate(AttendanceEndDate = ifelse(is.na(AttendanceEndDate), as.Date(DischargeDate), as.Date(AttendanceEndDate))) %>%
    mutate(LinkedDeathdate = as.Date(LinkedDeathdate)) %>%
    mutate(is_admitted = ifelse(is.na(AdmissionDate), F, T),
           is_icu = ifelse(SpellIncludesCriticalCareAdmission == 1, T, F),
           is_death = ifelse(LinkedDeathdate >= AttendanceStartDate &
                             LinkedDeathdate <= AttendanceEndDate, T, F))
#     mutate(is_icu = ifelse(is.na(is_icu), F, T))

# Admission duration
admit_attendance_df %>%
    filter(!is.na(AdmissionDate),
           DischargeDate != "NULL") %>%
    group_by(age_group) %>%
    summarise(median_duration = median(duration),
              duration_low = quantile(duration, probs = c(0.25)),
              duration_high = quantile(duration, probs = c(0.75)))
# Admission, ICU, death rate by age
admit_attendance_df %>%
    group_by(age_group) %>%
    summarise(admit_rate = round(sum(is_admitted, na.rm = T) / n() * 100, 1),
              icu_rate = round(sum(is_icu, na.rm = T) / n() * 100, 1),
              death_rate = round(sum(is_death, na.rm = T) / n() * 100, 2))
# Overall Admission, ICU, death rate
admit_attendance_df %>%
    summarise(admit_rate = round(sum(is_admitted, na.rm = T) / n() * 100, 1),
              icu_rate = round(sum(is_icu, na.rm = T) / n() * 100, 1),
              death_rate = round(sum(is_death, na.rm = T) / n() * 100, 2))

Joining with `by = join_by(ClusterID, AttendanceStartDate)`
Warning message in left_join(., admission_df):
“Each row in `x` is expected to match at most 1 row in `y`.
ℹ Row 18 of `x` matches multiple rows.
ℹ If multiple matches are expected, set `multiple = "all"` to silence this warning.”


age_group,median_duration,duration_low,duration_high
<chr>,<drtn>,<drtn>,<drtn>
7-15,1 days,1 days,2 days
<7,1 days,0 days,2 days
>15,2 days,1 days,7 days


age_group,admit_rate,icu_rate,death_rate
<chr>,<dbl>,<dbl>,<dbl>
7-15,13.2,0.3,0.01
<7,17.0,0.4,0.01
>15,34.3,1.5,0.41


admit_rate,icu_rate,death_rate
<dbl>,<dbl>,<dbl>
30.9,1.3,0.34


### Biomarker tests

In [11]:
test_names <- c("ALT", "AST", "ALK.PHOSPHATASE", "BILIRUBIN", "ALBUMIN", "GGT",
               "CRP", "WHITE CELLS")
tests <- admit_attendance_df %>%
    right_join(test_parsed) %>%
    filter(TestName %in% test_names)

Joining with `by = join_by(ClusterID, AttendanceStartDate)`
Warning message in right_join(., test_parsed):
“Each row in `x` is expected to match at most 1 row in `y`.
ℹ Row 2 of `x` matches multiple rows.
ℹ If multiple matches are expected, set `multiple = "all"` to silence this warning.”


In [16]:
test_summary <- admit_attendance_df %>%
    left_join(test_parsed) %>%
    group_by(ClusterID, AttendanceStartDate) %>%
    summarise(n_tests = n_distinct(TestName, na.rm = T),
              n_ALT = sum(TestName == "ALT", na.rm = T),
              n_AST = sum(TestName == "AST", na.rm = T)) %>%
    ungroup()

# Prop. of patients tested
test_summary %>%
    summarise(sum(n_tests > 0) / nrow(test_summary))

n_pat_with_tests <- deframe(test_summary %>%
    summarise(sum(n_tests > 0)))

# Prop. with ALT test
test_summary %>%
    summarise(sum(n_ALT > 0) / n_pat_with_tests)

# Prop. with AST test
test_summary %>%
    summarise(sum(n_AST > 0) / n_pat_with_tests)

Joining with `by = join_by(ClusterID, AttendanceStartDate)`
Warning message in left_join(., test_parsed):
“Each row in `x` is expected to match at most 1 row in `y`.
ℹ Row 2 of `x` matches multiple rows.
ℹ If multiple matches are expected, set `multiple = "all"` to silence this warning.”
`summarise()` has grouped output by 'ClusterID'. You can override using the `.groups` argument.


sum(n_tests > 0)/nrow(test_summary)
<dbl>
0.5880912


sum(n_ALT > 0)/n_pat_with_tests
<dbl>
0.9034954


sum(n_AST > 0)/n_pat_with_tests
<dbl>
0.0117504


In [14]:
test_df %>% distinct(TestName, RefRange, Units)

TestName,RefRange,Units
<chr>,<chr>,<chr>
ALBUMIN,32-50,g/L
CRP,0-5,mg/L
EOSINOPHILS,0.0-0.5,x10*9/L
WHITE CELLS,4.0-11.0,x10*9/L
PLATELETS,150-400,x10*9/L
ALK.PHOSPHATASE,30-130,IU/L
ALT,10-45,IU/L
MONOCYTES,0.2-1.0,x10*9/L
NEUTROPHILS,2.0-7.0,x10*9/L


In [8]:
for (test in test_names) {
    stat_temp <- tests %>%
        mutate(age_group = factor(age_group, c("<7", "7-15", ">15"))) %>%
        filter(TestName == test) %>%
        group_by(age_group) %>%
        summarise(median_test = median(Value),
                  test_low = quantile(Value, probs = c(0.25)),
                  test_high = quantile(Value, probs = c(0.75)))
    print(test)
    print(stat_temp)
}

[1] "ALT"
# A tibble: 3 × 4
  age_group median_test test_low test_high
  <fct>           <dbl>    <dbl>     <dbl>
1 <7                 15       12        21
2 7-15               14       11        20
3 >15                19       13        30
[1] "AST"
# A tibble: 3 × 4
  age_group median_test test_low test_high
  <fct>           <dbl>    <dbl>     <dbl>
1 <7                 41       31      144.
2 7-15               33       21      119 
3 >15                41       22      110.
[1] "ALK.PHOSPHATASE"
# A tibble: 3 × 4
  age_group median_test test_low test_high
  <fct>           <dbl>    <dbl>     <dbl>
1 <7                195      160       237
2 7-15              181      115       247
3 >15                79       64       101
[1] "BILIRUBIN"
# A tibble: 3 × 4
  age_group median_test test_low test_high
  <fct>           <dbl>    <dbl>     <dbl>
1 <7                  5        3         7
2 7-15                7        5        11
3 >15                 9        6        14
[1] "ALBUM

In [15]:
tests

ClusterID,AttendanceStartDate,AttendanceEndDate,LocationAttended,IMDScore,EthnicGroupCode,LinkedSex,LinkedBirthmonth,LinkedDeathdate,SpineCheckDate,⋯,is_icu,is_death,CollectionID,SetName,TestName,LLN,ULN,Units,Value,value_class
<int>,<date>,<dbl>,<chr>,<chr>,<chr>,<chr>,<date>,<date>,<chr>,⋯,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<fct>
2077289,2016-03-01,16861,J-ED,5.173,A,M,1946-03-01,2030-01-14,2023-01-11 14:00:09.793,⋯,TRUE,FALSE,N6918463,A2,ALK.PHOSPHATASE,30,130,IU/L,228.00,Mild (1-2x ULN)
2077289,2016-03-01,16861,J-ED,5.173,A,M,1946-03-01,2030-01-14,2023-01-11 14:00:09.793,⋯,TRUE,FALSE,N6918463,A2,ALT,10,45,IU/L,1967.00,Severe (>5x ULN)
2077289,2016-03-01,16861,J-ED,5.173,A,M,1946-03-01,2030-01-14,2023-01-11 14:00:09.793,⋯,TRUE,FALSE,N6918463,A2,BILIRUBIN,0,21,umol/L,27.00,Mild (1-2x ULN)
2077289,2016-03-01,16861,J-ED,5.173,A,M,1946-03-01,2030-01-14,2023-01-11 14:00:09.793,⋯,TRUE,FALSE,N6918462,FBC,WHITE CELLS,4.0,11,x10*9/L,9.52,Normal <= ULN
2077289,2016-03-01,16861,J-ED,5.173,A,M,1946-03-01,2030-01-14,2023-01-11 14:00:09.793,⋯,TRUE,FALSE,N6918463,A2,ALBUMIN,32,50,g/L,26.00,Normal <= ULN
2077289,2016-03-01,16861,J-ED,5.173,A,M,1946-03-01,2030-01-14,2023-01-11 14:00:09.793,⋯,TRUE,FALSE,N6918463,A2,CRP,0,5,mg/L,104.20,Severe (>5x ULN)
6834862,2016-03-01,16862,J-WD EAU,9.802,A,F,1924-02-01,2019-09-19,2021-12-11 11:00:06.790,⋯,TRUE,FALSE,N6910787,FBC,WHITE CELLS,4.0,11,x10*9/L,7.76,Normal <= ULN
6834862,2016-03-01,16862,J-WD EAU,9.802,A,F,1924-02-01,2019-09-19,2021-12-11 11:00:06.790,⋯,TRUE,FALSE,N6910788,A2,ALBUMIN,32,50,g/L,35.00,Normal <= ULN
6834862,2016-03-01,16862,J-WD EAU,9.802,A,F,1924-02-01,2019-09-19,2021-12-11 11:00:06.790,⋯,TRUE,FALSE,N6910788,A2,ALK.PHOSPHATASE,30,130,IU/L,57.00,Normal <= ULN


In [ ]:
for (test in test_names) {
    stat_temp <- tests %>%
        filter(adeno_pos) %>%
        mutate(age_group = factor(age_group, c("<7", "7-15", ">15"))) %>%
        filter(TestName == test) %>%
        group_by(age_group) %>%
        summarise(median_test = median(Value),
                  test_low = quantile(Value, probs = c(0.25)),
                  test_high = quantile(Value, probs = c(0.75)))
    print(test)
    print(stat_temp)
}

### Diagnostic codes

In [9]:
age_df %>%
    filter(ClusterID == 2561819)

ClusterID,AttendanceStartDate,AttendanceEndDate,LocationAttended,IMDScore,EthnicGroupCode,LinkedSex,LinkedBirthmonth,LinkedDeathdate,SpineCheckDate,LinkageWarningFlag,AttendanceMonth,AttendanceTwoMonth,age_upon_presentation,age_group
<int>,<date>,<date>,<chr>,<chr>,<chr>,<chr>,<date>,<chr>,<chr>,<int>,<date>,<date>,<dbl>,<chr>
2561819,2021-04-20,2021-04-20,J-ED,NULL,H,M,1991-07-01,NULL,2023-01-18 14:00:10.977,0,2021-04-01,2021-03-01,29,>15
2561819,2022-01-13,2022-01-13,J-WD L4 AAU,11.366,H,M,1991-07-01,NULL,2023-01-18 14:00:10.977,0,2022-01-01,2022-01-01,30,>15
2561819,2022-12-10,2022-12-10,J-WD L4 AAU,7.355,H,M,1991-07-01,NULL,2023-01-18 14:00:10.977,0,2022-12-01,2022-11-01,31,>15


In [10]:
total_count <- deframe(diag_df %>% 
                       distinct(ClusterID, AttendanceStartDate) %>%
                       nrow())
diag_df %>%
    left_join(age_df) %>%
    filter(!is.na(age_group)) %>%
    mutate(age_group = factor(age_group, c("<7", "7-15", ">15"))) %>%
    group_by(DiagCode, age_group) %>%
    summarise(n = n()) 


Joining with `by = join_by(ClusterID, AttendanceStartDate)`
Warning message in left_join(., age_df):
“Each row in `x` is expected to match at most 1 row in `y`.
ℹ Row 37 of `x` matches multiple rows.
ℹ If multiple matches are expected, set `multiple = "all"` to silence this warning.”
`summarise()` has grouped output by 'DiagCode'. You can override using the `.groups` argument.


DiagCode,age_group,n
<chr>,<fct>,<int>
B159,<7,2
B159,7-15,4
B159,>15,32
B162,>15,1
B169,>15,28
B171,>15,82
B172,>15,26
B178,7-15,1
B178,>15,5
